In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score

# Load the dataset
data = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')
data_submission = pd.read_csv('sample_submission.csv')

# Fill missing values
# Fill X2 (numerical) with its mean
data['X2'] = data['X2'].fillna(data['X2'].mean())
data_test['X2'] = data_test['X2'].fillna(data['X2'].mean())
# Fill X9 (categorical) with its mode
data['X9'] = data['X9'].fillna(data['X9'].mode()[0])
data_test['X9'] = data_test['X9'].fillna(data['X9'].mode()[0])

# Encode categorical columns
categorical_cols = data.select_dtypes(include=['object']).columns
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    # Fit on combined data to handle unseen categories
    combined_data = pd.concat([data[col], data_test[col]], axis=0)
    le.fit(combined_data)
    label_encoders[col] = le
    data[col] = le.transform(data[col])
    data_test[col] = le.transform(data_test[col])

# Select relevant features (X6, X7, X11) and target variable (Y)
selected_features = ['X6', 'X7', 'X11']
X = data[selected_features]  # Features
X_real_test = data_test[selected_features]  # Features
y = data['Y']  # Target

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardization for SVR
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_real_test_scaled = scaler.transform(X_real_test)

# Grid search for SVR hyperparameter tuning
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [0.001, 0.01, 0.1, 1, 'scale', 'auto']
}

grid_search = GridSearchCV(SVR(kernel='rbf'), param_grid, cv=5, scoring='r2')
grid_search.fit(X_train_scaled, y_train)

best_svr_model = grid_search.best_estimator_
best_svr_predictions = best_svr_model.predict(X_test_scaled)
best_svr_mse = mean_squared_error(y_test, best_svr_predictions)
best_svr_r2 = r2_score(y_test, best_svr_predictions)

# Display results
print(f"Best SVR Parameters: {grid_search.best_params_}")
print(f"Best SVR Mean Squared Error: {best_svr_mse}")
print(f"Best SVR R-Squared: {best_svr_r2}")

# Predict on the test dataset
test_predictions_simple = best_svr_model.predict(X_real_test_scaled)
simplified_submission = data_submission.copy()
simplified_submission['Y'] = test_predictions_simple

# Save the submission file
simplified_submission.to_csv('sample_submission.csv', index=False)
print("Simplified submission file created at 'sample_submission.csv'")


Best SVR Parameters: {'C': 100, 'gamma': 0.1}
Best SVR Mean Squared Error: 0.2955374953232189
Best SVR R-Squared: 0.7247510001882953
Simplified submission file created at 'sample_submission.csv'
